<a href="https://colab.research.google.com/github/rithikkulkarni/simstudent-data-preprocessing/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd

file_path = 'https://github.com/rithikkulkarni/simstudent-data-preprocessing/raw/main/datasets/Algebra2017May(YesPrep)-APLUS-SUBSET.xlsx'

df = pd.read_excel(file_path, sheet_name="Algebra2017May(YesPrep)-APLUS", engine="openpyxl")
print(df.shape)

(140943, 62)


In [44]:
### Loading the already filtered dataset from github
file_path = 'https://github.com/rithikkulkarni/simstudent-data-preprocessing/raw/main/datasets/SimStudent_StudyVII_May2017_Preprocessed_Subset.xlsx'

df = pd.read_excel(file_path)
print(df.shape)

(140943, 19)


In [68]:
unique_students = df['Anon Student Id'].nunique()
print(f'# of Unique Students: {unique_students}')

# of Unique Students: 266


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140943 entries, 0 to 140942
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Row                          140943 non-null  int64         
 1   Sample Name                  140943 non-null  object        
 2   Transaction Id               140943 non-null  object        
 3   Anon Student Id              140943 non-null  object        
 4   Session Id                   140943 non-null  object        
 5   Time                         140943 non-null  datetime64[ns]
 6   Time Zone                    140943 non-null  object        
 7   Duration (sec)               140943 non-null  object        
 8   Student Response Type        140943 non-null  object        
 9   Student Response Subtype     63907 non-null   object        
 10  Tutor Response Type          136474 non-null  object        
 11  Tutor Response Subtype    

In [70]:
### REMOVAL OF UNWANTED COLUMNS (RUN ONLY ONCE)
df = df.drop([
  'Anon Student Id',
  'Feedback Classification',
  'Total Num Hints',
  'Condition Type',
  'KC Category (Single-KC)',
  'KC Category (Unique-step)',
  'Unnamed: 60',
  'Unnamed: 61',
  'Row',
  'Sample Name',
  'Transaction Id',
  'School',
  'Time Zone',
  'Time',
  'Tutor Response Type',
  'Tutor Response Subtype',
  'Level (Unit)',
  'Problem Name',
  'KC (Problem)',
  'KC (Step)',
  'KC Category (Step)',
  'KC (ActualSkill)',
  'KC Category (ActualSkill)',
  'KC (Default)',
  'KC Category (Default)',
  'KC (ProblemSubmit)',
  'KC Category (ProblemSubmit)',
  'KC (Single-KC)',
  'KC (Unique-step)',
  'Class',
  'CF (ACTION_TYPE)',
  'CF (HINT_FOLLOWED)',
  'CF (HINT_SUBJECT)',
  'CF (HINT_TYPE)',
  'CF (INFO)',
  'CF (STUDENT_LOG_COUNT)',
  'CF (date)',
  'Participation Day Key',
  'Student Response Subtype',
  'Problem Start Time'
], axis=1)

In [71]:
# Renaming columns in preparation for dataset export
df = df.rename(columns={
  'CF (USERID)': 'user_id',
  'Session Id': 'session_id',
  'Duration (sec)': 'duration_sec',
  'Student Response Type': 'student_response_type',
  'Problem View': 'problem_view',
  'Attempt At Step': 'attempt_at_step',
  'Is Last Attempt': 'is_last_attempt',
  'Outcome': 'outcome',
  'Input': 'input',
  'Feedback Text': 'feedback_text',
  'Help Level': 'help_level',
  'Condition Name': 'condition_name',
  'Step Name': 'step_name',
  'Selection': 'selection',
  'Action': 'action',
  'KC Category (Problem)': 'problem_category',
  'CF (ACTION)': 'action_explanation',
  'CF (DURATION)': 'duration',
  'CF (RESULT)': 'result',
  'CF (STATUS)': 'status',
  'Participation Day': 'participation_day',
  'CF (tool_event_time)': 'tool_event_time'
})

# Look at CF (RESULT) with CF (ACTION)
# Example: left tutor tab and switched to quiz tab

In [72]:
# Get a visual of the amount of unique values in each column

for col in df.columns:
  print(f'{col}: {df[col].nunique()}')

session_id: 856
duration_sec: 1100
student_response_type: 2
problem_view: 25
step_name: 1702
attempt_at_step: 385
is_last_attempt: 2
outcome: 4
selection: 160
action: 5
input: 1944
feedback_text: 8253
help_level: 4
condition_name: 2
problem_category: 8
action_explanation: 58
duration: 2242
result: 2946
status: 3
user_id: 264
participation_day: 4
tool_event_time: 140041


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140943 entries, 0 to 140942
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   session_id             140943 non-null  object 
 1   duration_sec           140943 non-null  object 
 2   student_response_type  140943 non-null  object 
 3   problem_view           140943 non-null  int64  
 4   step_name              136474 non-null  object 
 5   attempt_at_step        136474 non-null  float64
 6   is_last_attempt        136474 non-null  float64
 7   outcome                136474 non-null  object 
 8   selection              68958 non-null   object 
 9   action                 68958 non-null   object 
 10  input                  68954 non-null   object 
 11  feedback_text          20420 non-null   object 
 12  help_level             5806 non-null    float64
 13  condition_name         140943 non-null  object 
 14  problem_category       86862 non-nul

In [51]:
df.to_excel('SimStudent_StudyVII_May2017_Preprocessed_Sorted_Subset.xlsx', index=False)

# Filtering done. Now for sorting and visualizing

In [ ]:
### Convert UTC times to EST


In [66]:
### Convert UTC times to EST

import pytz
from datetime import datetime

utc = pytz.UTC
eastern = pytz.timezone("US/Eastern")

def convert_time(value):
    try:
        if pd.isna(value):  # check for NaT or None
            return pd.NaT

        value_str = str(value)

        if "UTC" in value_str:
            dt = datetime.strptime(value_str.replace(" UTC", ""), "%Y-%m-%d %H:%M:%S.%f")
            dt = utc.localize(dt).astimezone(eastern)
        else:
            dt = datetime.strptime(value_str, "%Y-%m-%d-%H:%M:%S.%f")
            dt = eastern.localize(dt)

        return dt

    except Exception as e:
        print(f"Failed to parse: {value} → {e}")
        return pd.NaT

def format_datetime_for_output(dt):
    if pd.isna(dt):
        return ""
    if hasattr(dt, 'tzinfo'):
        dt = dt.tz_localize(None)  # remove timezone
    return dt.strftime("%Y-%m-%d-%H:%M:%S.%f")


df['tool_event_time_formatted_1'] = df['tool_event_time'].apply(format_datetime_for_output)
df['tool_event_time_formatted_2'] = df['tool_event_time'].apply(convert_time)
df['tool_event_time_formatted_3'] = df['tool_event_time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

### Create sorted dataframe
df.sort_values(by=['user_id','tool_event_time_formatted_3'], inplace=True)


ValueError: NaTType does not support strftime

In [49]:
df['tool_event_time'] = df['tool_event_time_str']
df = df.drop('tool_event_time_str', axis=1)

In [65]:
df.head(50)

,session_id,duration_sec,student_response_type,problem_view,step_name,attempt_at_step,is_last_attempt,outcome,selection,action,...,duration,result,status,user_id,participation_day,tool_event_time,tool_event_time_str,tool_event_time_formatted_1,tool_event_time_formatted_2,tool_event_time_formatted_3
67939,Lba74e43f-f63f-40b2-82c6-88b04fd58925,2,HINT_REQUEST,1,Unknown,1.0,0.0,UNGRADED,NaN,NaN,...,NaN,NaN,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
67940,Lba74e43f-f63f-40b2-82c6-88b04fd58925,12,HINT_REQUEST,1,'START,1.0,0.0,UNGRADED,NaN,NaN,...,NaN,NaN,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
72293,Lba74e43f-f63f-40b2-82c6-88b04fd58925,.,HINT_REQUEST,1,'START,2.0,0.0,UNGRADED,NaN,NaN,...,NaN,NaN,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
72297,Lba74e43f-f63f-40b2-82c6-88b04fd58925,.,HINT_REQUEST,1,Unknown,2.0,0.0,UNGRADED,NaN,NaN,...,1903.0,barbie,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
72298,Lba74e43f-f63f-40b2-82c6-88b04fd58925,.,HINT_REQUEST,1,Unknown,3.0,0.0,UNGRADED,NaN,NaN,...,NaN,NaN,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
72742,Lba74e43f-f63f-40b2-82c6-88b04fd58925,93,HINT_REQUEST,1,Unknown,4.0,0.0,UNGRADED,NaN,NaN,...,NaN,NaN,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
76725,Lba74e43f-f63f-40b2-82c6-88b04fd58925,285.5,HINT_REQUEST,1,Unknown,5.0,0.0,UNGRADED,NaN,NaN,...,2557.0,Tutor,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
76726,Lba74e43f-f63f-40b2-82c6-88b04fd58925,285.5,HINT_REQUEST,1,Unknown,6.0,0.0,UNGRADED,NaN,NaN,...,NaN,Quiz,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
76751,Lba74e43f-f63f-40b2-82c6-88b04fd58925,0.5,ATTEMPT,1,'START,3.0,0.0,INCORRECT,Quiz,TabClicked,...,NaN,simst-launch-take-quiz_Quiz,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
76755,Lba74e43f-f63f-40b2-82c6-88b04fd58925,0.5,ATTEMPT,1,'START,4.0,0.0,CORRECT,Quiz,TabClicked,...,NaN,simst-tab-clicked,Start,AAAEV,1.0,NaT,NaN,,NaT,NaN
